In [1]:
import pandas as pd
import numpy as np


tourney_seeds = pd.read_csv("./Data/TourneySeeds.csv")
rank = []
teams = []
for seed, team in tourney_seeds.loc[:,['Seed','Team']].values:
    rank.append(int(seed[1:3]))
tourney_seeds["NumericalSeeds"] = rank

In [4]:
tourney_compact_results = pd.read_csv("./Data/TourneyCompactResults.csv")

In [17]:
tourney_seeds.head()

,Season,Seed,Team,NumericalSeeds
0,1985,W01,1207,1
1,1985,W02,1210,2
2,1985,W03,1228,3
3,1985,W04,1260,4
4,1985,W05,1374,5


In [3]:
tourney_compact_results.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [5]:
result = pd.DataFrame({
        "Season": pd.concat([tourney_compact_results['Season'], tourney_compact_results['Season']], ignore_index=True),
        "Player": pd.concat([tourney_compact_results['Wteam'], tourney_compact_results['Lteam']], ignore_index=True),
        "Opponent": pd.concat([tourney_compact_results['Lteam'], tourney_compact_results['Wteam']], ignore_index=True),
        "Result": np.hstack([np.ones(tourney_compact_results.shape[0], dtype=int), np.zeros(tourney_compact_results.shape[0], dtype=int)])
    }, columns=["Result", "Season", "Player", "Opponent"])

In [6]:
t = pd.merge(result, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t[['Result', 'Season','Player', 'Opponent', 'NumericalSeeds']]
t.columns=['Result', 'Season','Player', 'Opponent', 'PlayerSeed']
result = t

In [7]:
t = pd.merge(result, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t[['Result', 'Season', 'Player', 'Opponent', 'PlayerSeed', 'NumericalSeeds']]
t.columns = ['Result', 'Season','Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
result = t

In [6]:
result.shape

(4100, 6)

In [8]:
regular = pd.read_csv("./Data/RegularSeasonDetailedResults.csv")
regular.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [9]:
regular_rev = regular.ix[:, [0,1,4,5,2,3,7,21,22,23,24,25,26,27,28,29,30,31,32,33,8,9,10,11,12,13,14,15,16,17,18,19,20]]
new_regular = pd.DataFrame(np.vstack([regular_rev.values, 
                                   regular.ix[:, [0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]].values], 
                                 ), columns = ["Season","Daynum","Player","PlayerScore","Opponent","OpponentScore","Numot","Wfgm","Wfga","Wfgm3","Wfga3","Wftm","Wfta","Wor","Wdr","Wast","Wto","Wstl","Wblk","Wpf","Lfgm","Lfga","Lfgm3","Lfga3","Lftm","Lfta","Lor","Ldr","Last","Lto","Lstl","Lblk","Lpf"])

In [10]:
loc = []
revloc = []
for itm in regular['Wloc'].values:
    if itm=='N':
        loc.append(0)
        revloc.append(0)
    elif itm=='H':
        loc.append(1)
        revloc.append(2)
    else:
        loc.append(2)
        revloc.append(1)
new_regular['PlayerLoc']=np.hstack([revloc, loc])
new_regular['PlayerResult']=np.hstack([np.zeros(regular.shape[0], dtype=np.int), np.ones(regular.shape[0], dtype=np.int)])

generate weight

In [11]:
w = np.ones(new_regular.shape[0])
#w[new_regular['Daynum'].values>=(132-7)]=2.0
new_regular['Weight']=w

Effective field goal percentage

In [12]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfgm', 'Wfgm3', 'Wfga', 'Lfgm', 'Lfgm3', 'Lfga', 'Weight']].copy()
t["EfgO"] = np.require(t['Wfgm'].values, dtype=np.float)+np.require(t['Wfgm3'].values, dtype=np.float)/2
t["EfgO"] = t['EfgO']/t['Wfga']*t['Weight']
t["EfgD"] = np.require(t['Lfgm'].values, dtype=np.float)+np.require(t['Lfgm3'].values, dtype=np.float)/2
t["EfgD"] = t['EfgD']/t['Lfga']*t['Weight']
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'EfgO': t['EfgO'], 'EfgD': t['EfgD']}, columns=['EfgO', 'EfgD'])
rst.head()

EfgO      EfgD
Season Player                    
2003   1102    0.584407  0.514142
       1103    0.536564  0.549098
       1104    0.475785  0.480099
       1105    0.457983  0.509308
       1106    0.481697  0.452774

In [13]:
feature = pd.DataFrame(tourney_seeds[['Season', 'Team']].values, columns=['Season', 'Player'])
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

Turnover percentage

In [14]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfga', 'Wfta', 'Wto', 'Lfga', 'Lfta', 'Lto', 'Weight']].copy()
t["TopO"] = np.require(t['Wfga'].values+t['Wto'].values, dtype=np.float) + \
0.44* np.require(t['Wfta'].values, dtype=np.float)
t["TopO"] = t['Wto']/t['TopO']*t['Weight']
t["TopD"] = np.require(t['Lfga'].values+t['Lto'].values, dtype=np.float) + \
0.44* np.require(t['Lfta'].values, dtype=np.float)
t["TopD"] = t['Lto']/t['TopD']*t['Weight']
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'TopO': t['TopO'], 'TopD': t['TopD']}, columns=['TopO', 'TopD'])
rst.head()

TopO      TopD
Season Player                    
2003   1102    0.193121  0.203236
       1103    0.159805  0.186897
       1104    0.167275  0.179498
       1105    0.207792  0.212924
       1106    0.213721  0.194026

In [15]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

Rebounding

In [16]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wor', 'Wdr', 'Lor', 'Ldr', 'Weight']].copy()
t["ORB"] = np.require(t["Wor"].values+t["Ldr"].values, dtype=np.float)
t["ORB"] = t['Wor']/t['ORB']*t['Weight']
t["DRB"] = np.require(t["Wdr"].values+t["Lor"].values, dtype=np.float)
t["DRB"] = t['Wdr']/t['DRB']*t['Weight']
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'ORB': t['ORB'], 'DRB': t['DRB']}, columns=['ORB', 'DRB'])
rst.head()

ORB       DRB
Season Player                    
2003   1102    0.168235  0.630384
       1103    0.305803  0.626998
       1104    0.371256  0.686897
       1105    0.335166  0.641434
       1106    0.349480  0.679342

In [17]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

free throws

In [18]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfga', 'Wftm', 'Lfga', 'Lftm', 'Weight']].copy()
t['OFT'] = np.require(t['Wftm'].values, dtype=np.float)/np.require(t['Wfga'].values, dtype=np.float)*t['Weight'].values
t['DFT'] = np.require(t['Lftm'].values, dtype=np.float)/np.require(t['Lfga'].values, dtype=np.float)*t['Weight'].values
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'OFT': t['OFT'], 'DFT': t['DFT']}, 
                   columns=['OFT', 'DFT'])
rst.head()

OFT       DFT
Season Player                    
2003   1102    0.291436  0.336057
       1103    0.342976  0.282092
       1104    0.263855  0.223414
       1105    0.255629  0.289471
       1106    0.199740  0.297459

In [19]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

assist

In [20]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfga', 'Wfta', 'Wast', 'Lfga', 'Lfta', 'Last', 'Weight']].copy()
t["AstO"] = np.require(t['Wfga'].values+t['Wast'].values, dtype=np.float) + \
0.44* np.require(t['Wfta'].values, dtype=np.float)
t["AstO"] = t['Wast']/t['AstO']*t['Weight']
t["AstD"] = np.require(t['Lfga'].values+t['Last'].values, dtype=np.float) + \
0.44* np.require(t['Lfta'].values, dtype=np.float)
t["AstD"] = t['Last']/t['AstD']*t['Weight']
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'AstO': t['AstO'], 'AstD': t['AstD']}, columns=['AstO', 'AstD'])
rst.head()

AstO      AstD
Season Player                    
2003   1102    0.211098  0.151343
       1103    0.183910  0.188268
       1104    0.152915  0.155161
       1105    0.168060  0.183142
       1106    0.155741  0.155688

In [21]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

In [22]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wto', 'Weight']].copy()
t = t.groupby(by=['Season', 'Player']).sum()
rst = pd.DataFrame({'FGRate': t['Wfgm']/t['Wfga'], 'FG3Rate': t['Wfgm3']/t['Wfga3'], 'FTRate': t['Wftm']/t['Wfta'], 'AstTo': t['Wast']/t['Wto']}, 
                   columns=['FGRate', 'FG3Rate', 'FTRate', 'AstTo'])
rst.head()

FGRate   FG3Rate    FTRate     AstTo
Season Player                                        
2003   1102    0.481149  0.375643  0.651357  1.137500
       1103    0.486074  0.338710  0.736390  1.205279
       1104    0.420362  0.320144  0.709898  0.911290
       1105    0.395755  0.364815  0.705986  0.779381
       1106    0.423773  0.346154  0.646421  0.685535

In [23]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

Score Ranking

In [24]:
w = np.ones(regular.shape[0])
w[regular['Daynum'].values>=(132-15)]=2.0
regular['Weight']=w

In [25]:
from rankit.util import Converter
from rankit.ranker import *

frames = []
for year in range(2003, 2017):
    table = regular[regular['Season']==year][['Wteam', 'Lteam', 'Wscore', 'Lscore', 'Weight']].copy()
    table.columns = [['primary', 'secondary', 'rate1', 'rate2', 'weight']]
    #table['weight'] = np.ones(table.shape[0])
    
    cvt = Converter(table=table)
    M=cvt.MasseyMatrix()
    b=cvt.MasseyVector()
    ranker=MasseyRank(itemlist=cvt.ItemList())
    x=ranker.rate(M,b)
    bigtable=ranker.rank(x)
    bigtable.drop(['rank'], 1, inplace=True)
    bigtable = bigtable.rename(columns={'rate': "Massey"})
    
    C=cvt.ColleyMatrix()
    b=cvt.ColleyVector()
    ranker=ColleyRank(itemlist=cvt.ItemList())
    x=ranker.rate(C,b)
    r=ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "Colley"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    M=cvt.SymmetricDifferenceMatrix()
    ranker=DifferenceRank(itemlist=cvt.ItemList())
    x=ranker.rate(M)
    r=ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "Differ"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    M=cvt.RateDifferenceVoteMatrix()
    ranker=MarkovRank(itemlist=cvt.ItemList(), epsilon=0.6)
    x = ranker.rate(M)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "MarkovRDV"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    M=cvt.SimpleDifferenceVoteMatrix()
    ranker=MarkovRank(itemlist=cvt.ItemList(), epsilon=0.6)
    x = ranker.rate(M)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "MarkovSDV"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    M=cvt.RateVoteMatrix()
    ranker=ODRank(itemlist=cvt.ItemList(), iteration=1000000)
    x=ranker.rate(M)
    r=ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "OD"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    C = cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-4)
    x = ranker.rate(M.T, C)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "Keener"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    bigtable['Team']=np.require(bigtable.ix[:, 0].values, dtype=np.int)
    bigtable.drop('title', 1, inplace=True)
    bigtable['Year'] = year*np.ones(bigtable.shape[0], dtype=np.int)
    frames.append(bigtable)

/home/ike/Projects/ranking/rankit/util/converter.py:202: RuntimeWarning: invalid value encountered in divide
  S = D/C
/home/ike/Projects/ranking/rankit/ranker/EigRank.py:27: RuntimeWarning: invalid value encountered in divide
  S = S/np.expand_dims(s, axis=1)


In [26]:
ranking = pd.concat(frames, ignore_index=True)
feature = pd.merge(feature, ranking, left_on=["Season", "Player"], right_on=["Year", "Team"])
feature.drop(['Year','Team'], axis=1, inplace=True)
feature.head()

,Season,Player,EfgO,EfgD,TopO,TopD,ORB,DRB,OFT,DFT,...,FG3Rate,FTRate,AstTo,Massey,Colley,Differ,MarkovRDV,MarkovSDV,OD,Keener
0,2003,1328,0.512124,0.448150,0.155078,0.181439,0.347284,0.709854,0.234856,0.236844,...,0.393673,0.707885,1.200565,50.711750,0.978662,0.940367,0.007066,0.006171,2982.190430,0.003958
1,2003,1448,0.511972,0.463124,0.178941,0.167749,0.429724,0.687237,0.354094,0.221934,...,0.344358,0.752591,0.965753,48.587179,0.940367,0.862895,0.007634,0.005863,2835.567139,0.003220
2,2003,1393,0.515151,0.445717,0.160408,0.165338,0.385242,0.630790,0.274586,0.199656,...,0.330435,0.693431,1.098734,48.047815,0.937698,0.801223,0.005666,0.005944,2810.180420,0.003929
3,2003,1257,0.528861,0.450485,0.156505,0.205507,0.356053,0.664037,0.292504,0.329973,...,0.360399,0.695883,1.241895,51.215123,0.946595,1.198777,0.011890,0.007738,2919.098877,0.004260
4,2003,1280,0.521359,0.463462,0.197467,0.208120,0.383446,0.695785,0.220035,0.199428,...,0.343621,0.676749,0.927505,49.042847,0.834543,0.816004,0.006985,0.005437,2905.012695,0.003999


In [27]:
# rebound

frames = []
for year in range(2003, 2017):
    table = regular[regular['Season']==year]
    t1 = pd.DataFrame({'primary': table['Wteam'],
                      'secondary': table['Lteam'],
                      'rate1': table['Wor'],
                      'rate2': table['Lor'],
                      'weight': table['Weight']})
    
    cvt = Converter(t1)
    
    M=cvt.RateVoteMatrix() 
    C=cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
    x = ranker.rate(M.T, C)
    bigtable = ranker.rank(x)
    bigtable.drop(['rank'], 1, inplace=True)
    bigtable.rename(columns={'rate': "KeenerOR"}, inplace=True)
    
    t2 = pd.DataFrame({'primary': table['Wteam'],
                      'secondary': table['Lteam'],
                      'rate1': table['Wdr'],
                      'rate2': table['Ldr'],
                      'weight': table['Weight']})
    
    cvt = Converter(t2)
    
    M=cvt.RateVoteMatrix() 
    C=cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
    x = ranker.rate(M.T, C)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "KeenerDR"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    t3 = pd.DataFrame({'primary': table['Wteam'],
                      'secondary': table['Lteam'],
                      'rate1': table['Wast'],
                      'rate2': table['Last'],
                      'weight': table['Weight']})
    
    cvt = Converter(t3)
    
    M=cvt.RateVoteMatrix() 
    C=cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
    x = ranker.rate(M.T, C)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "KeenerAST"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    t4 = pd.DataFrame({'primary': table['Wteam'],
                      'secondary': table['Lteam'],
                      'rate1': table['Wto'],
                      'rate2': table['Lto'],
                      'weight': table['Weight']})
    
    cvt = Converter(t4)
    
    M=cvt.RateVoteMatrix() 
    C=cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
    x = ranker.rate(M.T, C)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "KeenerTO"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    t5 = pd.DataFrame({'primary': table['Wteam'],
                      'secondary': table['Lteam'],
                      'rate1': table['Wstl'],
                      'rate2': table['Lstl'],
                      'weight': table['Weight']})
    
    cvt = Converter(t5)
    
    M=cvt.RateVoteMatrix() 
    C=cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
    x = ranker.rate(M.T, C)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "KeenerSTL"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    t6 = pd.DataFrame({'primary': table['Wteam'],
                      'secondary': table['Lteam'],
                      'rate1': table['Wblk'],
                      'rate2': table['Lblk'],
                      'weight': table['Weight']})
    
    cvt = Converter(t6)
    
    M=cvt.RateVoteMatrix() 
    C=cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
    x = ranker.rate(M.T, C)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "KeenerBLK"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    t7 = pd.DataFrame({'primary': table['Wteam'],
                      'secondary': table['Lteam'],
                      'rate1': table['Wpf'],
                      'rate2': table['Lpf'],
                      'weight': table['Weight']})
    
    cvt = Converter(t7)
    
    M=cvt.RateVoteMatrix() 
    C=cvt.CountMatrix()
    ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
    x = ranker.rate(M.T, C)
    r = ranker.rank(x)
    r.drop(['rank'], 1, inplace=True)
    r.rename(columns={'rate': "KeenerPF"}, inplace=True)
    bigtable = pd.merge(bigtable, r, on='title')
    
    bigtable['Team']=np.require(bigtable.ix[:, 0].values, dtype=np.int)
    bigtable.drop('title', 1, inplace=True)
    bigtable['Year'] = year*np.ones(bigtable.shape[0], dtype=np.int)
    frames.append(bigtable)

In [28]:
ranking = pd.concat(frames, ignore_index=True)

In [29]:
feature = pd.merge(feature, ranking, left_on=["Season", "Player"], right_on=["Year", "Team"])
feature.drop(['Year','Team'], axis=1, inplace=True)
feature.head()

,Season,Player,EfgO,EfgD,TopO,TopD,ORB,DRB,OFT,DFT,...,MarkovSDV,OD,Keener,KeenerOR,KeenerDR,KeenerAST,KeenerTO,KeenerSTL,KeenerBLK,KeenerPF
0,2003,1328,0.512124,0.448150,0.155078,0.181439,0.347284,0.709854,0.234856,0.236844,...,0.006171,2982.190430,0.003958,0.003897,0.004068,0.004310,0.003290,0.004268,0.004761,0.003570
1,2003,1448,0.511972,0.463124,0.178941,0.167749,0.429724,0.687237,0.354094,0.221934,...,0.005863,2835.567139,0.003220,0.003343,0.003473,0.003124,0.002880,0.002841,0.003573,0.002624
2,2003,1393,0.515151,0.445717,0.160408,0.165338,0.385242,0.630790,0.274586,0.199656,...,0.005944,2810.180420,0.003929,0.003473,0.004141,0.003825,0.003410,0.004474,0.005948,0.003349
3,2003,1257,0.528861,0.450485,0.156505,0.205507,0.356053,0.664037,0.292504,0.329973,...,0.007738,2919.098877,0.004260,0.004145,0.004132,0.004759,0.003171,0.004624,0.005179,0.003789
4,2003,1280,0.521359,0.463462,0.197467,0.208120,0.383446,0.695785,0.220035,0.199428,...,0.005437,2905.012695,0.003999,0.004004,0.004236,0.004791,0.003555,0.004165,0.004609,0.003582


## Feature normalization

In [30]:
from scipy import stats

cdf = dict()
hist, bin_edges = np.histogram(feature['EfgO'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['EfgO'] = stats.rv_discrete(name='EfgO', values=(bin_edges[:-1], hist))
feature['EfgO'] = cdf['EfgO'].cdf(feature['EfgO'].values)

In [31]:
hist, bin_edges = np.histogram(feature['EfgD'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['EfgD'] = stats.rv_discrete(name='EfgD', values=(bin_edges[:-1], hist))
feature['EfgD'] = cdf['EfgD'].cdf(feature['EfgD'].values)

hist, bin_edges = np.histogram(feature['TopO'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['TopO'] = stats.rv_discrete(name='TopO', values=(bin_edges[:-1], hist))
feature['TopO'] = cdf['TopO'].cdf(feature['TopO'].values)

hist, bin_edges = np.histogram(feature['TopD'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['TopD'] = stats.rv_discrete(name='TopD', values=(bin_edges[:-1], hist))
feature['TopD'] = cdf['TopD'].cdf(feature['TopD'].values)

hist, bin_edges = np.histogram(feature['ORB'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['ORB'] = stats.rv_discrete(name='ORB', values=(bin_edges[:-1], hist))
feature['ORB'] = cdf['ORB'].cdf(feature['ORB'].values)

hist, bin_edges = np.histogram(feature['DRB'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['DRB'] = stats.rv_discrete(name='DRB', values=(bin_edges[:-1], hist))
feature['DRB'] = cdf['DRB'].cdf(feature['DRB'].values)

hist, bin_edges = np.histogram(feature['OFT'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['OFT'] = stats.rv_discrete(name='DRB', values=(bin_edges[:-1], hist))
feature['OFT'] = cdf['OFT'].cdf(feature['OFT'].values)

hist, bin_edges = np.histogram(feature['DFT'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['DFT'] = stats.rv_discrete(name='DFT', values=(bin_edges[:-1], hist))
feature['DFT'] = cdf['DFT'].cdf(feature['DFT'].values)

hist, bin_edges = np.histogram(feature['AstO'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['AstO'] = stats.rv_discrete(name='AstO', values=(bin_edges[:-1], hist))
feature['AstO'] = cdf['AstO'].cdf(feature['AstO'].values)

hist, bin_edges = np.histogram(feature['AstD'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['AstD'] = stats.rv_discrete(name='AstD', values=(bin_edges[:-1], hist))
feature['AstD'] = cdf['AstD'].cdf(feature['AstD'].values)

hist, bin_edges = np.histogram(feature['FGRate'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['FGRate'] = stats.rv_discrete(name='FGRate', values=(bin_edges[:-1], hist))
feature['FGRate'] = cdf['FGRate'].cdf(feature['FGRate'].values)

hist, bin_edges = np.histogram(feature['FG3Rate'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['FG3Rate'] = stats.rv_discrete(name='FG3Rate', values=(bin_edges[:-1], hist))
feature['FG3Rate'] = cdf['FG3Rate'].cdf(feature['FG3Rate'].values)

hist, bin_edges = np.histogram(feature['FTRate'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['FTRate'] = stats.rv_discrete(name='FTRate', values=(bin_edges[:-1], hist))
feature['FTRate'] = cdf['FTRate'].cdf(feature['FTRate'].values)

hist, bin_edges = np.histogram(feature['AstTo'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['AstTo'] = stats.rv_discrete(name='AstTo', values=(bin_edges[:-1], hist))
feature['AstTo'] = cdf['AstTo'].cdf(feature['AstTo'].values)

In [32]:
feature.head()

,Season,Player,EfgO,EfgD,TopO,TopD,ORB,DRB,OFT,DFT,...,MarkovSDV,OD,Keener,KeenerOR,KeenerDR,KeenerAST,KeenerTO,KeenerSTL,KeenerBLK,KeenerPF
0,2003,1328,0.396552,0.164871,0.307112,0.545259,0.548491,0.747845,0.106681,0.506466,...,0.006171,2982.190430,0.003958,0.003897,0.004068,0.004310,0.003290,0.004268,0.004761,0.003570
1,2003,1448,0.394397,0.362069,0.784483,0.299569,0.994612,0.456897,0.964440,0.351293,...,0.005863,2835.567139,0.003220,0.003343,0.003473,0.003124,0.002880,0.002841,0.003573,0.002624
2,2003,1393,0.447198,0.134698,0.410560,0.270474,0.875000,0.024784,0.462284,0.169181,...,0.005944,2810.180420,0.003929,0.003473,0.004141,0.003825,0.003410,0.004474,0.005948,0.003349
3,2003,1257,0.664871,0.188578,0.340517,0.851293,0.635776,0.173491,0.653017,0.971983,...,0.007738,2919.098877,0.004260,0.004145,0.004132,0.004759,0.003171,0.004624,0.005179,0.003789
4,2003,1280,0.540948,0.370690,0.947198,0.884698,0.864224,0.567888,0.044181,0.165948,...,0.005437,2905.012695,0.003999,0.004004,0.004236,0.004791,0.003555,0.004165,0.004609,0.003582


In [33]:
result['SeedDelta'] = (result['OpponentSeed']-result['PlayerSeed']+16.0)/31
result = pd.merge(result, feature, on=['Season', 'Player'])
result = pd.merge(result, feature, left_on=['Season', 'Opponent'], right_on=['Season', 'Player'], suffixes=["Ply", "Opp"])

In [34]:
result.drop(['PlayerOpp'], 1, inplace=True)
result = result.rename(columns={"PlayerPly": "Player"})

In [32]:
result.head()

,Result,Season,Player,Opponent,PlayerSeed,OpponentSeed,SeedDelta,EfgOPly,EfgDPly,TopOPly,...,FG3RateOpp,FTRateOpp,AstToOpp,MasseyOpp,ColleyOpp,DifferOpp,MarkovRDVOpp,MarkovSDVOpp,ODOpp,KeenerOpp
0,1,2003,1421,1411,16,16,0.516129,0.122845,0.970905,0.945043,...,0.067888,0.010776,0.148707,83.157631,0.385637,0.062691,0.002100,0.002813,1954.497803,0.002251
1,0,2003,1421,1400,16,1,0.032258,0.122845,0.970905,0.945043,...,0.349138,0.604526,0.411638,111.175331,0.948126,0.775229,0.007392,0.005941,2920.588135,0.004127
2,0,2003,1163,1400,5,1,0.387097,0.581897,0.049569,0.822198,...,0.349138,0.604526,0.411638,111.175331,0.948126,0.775229,0.007392,0.005941,2920.588135,0.004127
3,0,2003,1277,1400,7,1,0.322581,0.265086,0.270474,0.918103,...,0.349138,0.604526,0.411638,111.175331,0.948126,0.775229,0.007392,0.005941,2920.588135,0.004127
4,0,2003,1345,1400,9,1,0.258065,0.060345,0.437500,0.709052,...,0.349138,0.604526,0.411638,111.175331,0.948126,0.775229,0.007392,0.005941,2920.588135,0.004127


In [35]:
result['MasseyDelta'] = result.MasseyPly-result.MasseyOpp
result.drop(['MasseyPly', 'MasseyOpp'], 1, inplace=True)
result['ColleyDelta'] = result.ColleyPly-result.ColleyOpp
result.drop(['ColleyPly', 'ColleyOpp'], 1, inplace=True)
result['DifferDelta'] = result.DifferPly-result.DifferOpp
result.drop(['DifferPly', 'DifferOpp'], 1, inplace=True)
result['MarkovRDVDelta'] = result.MarkovRDVPly-result.MarkovRDVOpp
result.drop(['MarkovRDVPly', 'MarkovRDVOpp'], 1, inplace=True)
result['MarkovSDVDelta'] = result.MarkovSDVPly-result.MarkovSDVOpp
result.drop(['MarkovSDVPly', 'MarkovSDVOpp'], 1, inplace=True)
result['ODDelta'] = result.ODPly-result.ODOpp
result.drop(['ODPly', 'ODOpp'], 1, inplace=True)
result['KeenerDelta'] = result.KeenerPly-result.KeenerOpp
result.drop(['KeenerPly', 'KeenerOpp'], 1, inplace=True)

result['KeenerORDelta'] = result.KeenerORPly-result.KeenerOROpp
result.drop(['KeenerORPly', 'KeenerOROpp'], 1, inplace=True)
result['KeenerDRDelta'] = result.KeenerDRPly-result.KeenerDROpp
result.drop(['KeenerDRPly', 'KeenerDROpp'], 1, inplace=True)
result['KeenerASTDelta'] = result.KeenerASTPly-result.KeenerASTOpp
result.drop(['KeenerASTPly', 'KeenerASTOpp'], 1, inplace=True)
result['KeenerTODelta'] = result.KeenerTOPly-result.KeenerTOOpp
result.drop(['KeenerTOPly', 'KeenerTOOpp'], 1, inplace=True)
result['KeenerSTLDelta'] = result.KeenerSTLPly-result.KeenerSTLOpp
result.drop(['KeenerSTLPly', 'KeenerSTLOpp'], 1, inplace=True)
result['KeenerBLKDelta'] = result.KeenerBLKPly-result.KeenerBLKOpp
result.drop(['KeenerBLKPly', 'KeenerBLKOpp'], 1, inplace=True)
result['KeenerPFDelta'] = result.KeenerPFPly-result.KeenerPFOpp
result.drop(['KeenerPFPly', 'KeenerPFOpp'], 1, inplace=True)

In [36]:
hist, bin_edges = np.histogram(result['MasseyDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['MasseyDelta'] = stats.rv_discrete(name='MasseyDelta', values=(bin_edges[:-1], hist))
result['MasseyDelta'] = cdf['MasseyDelta'].cdf(result['MasseyDelta'].values)

hist, bin_edges = np.histogram(result['ColleyDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['ColleyDelta'] = stats.rv_discrete(name='ColleyDelta', values=(bin_edges[:-1], hist))
result['ColleyDelta'] = cdf['ColleyDelta'].cdf(result['ColleyDelta'].values)

hist, bin_edges = np.histogram(result['DifferDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['DifferDelta'] = stats.rv_discrete(name='DifferDelta', values=(bin_edges[:-1], hist))
result['DifferDelta'] = cdf['DifferDelta'].cdf(result['DifferDelta'].values)

hist, bin_edges = np.histogram(result['MarkovRDVDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['MarkovRDVDelta'] = stats.rv_discrete(name='MarkovRDVDelta', values=(bin_edges[:-1], hist))
result['MarkovRDVDelta'] = cdf['MarkovRDVDelta'].cdf(result['MarkovRDVDelta'].values)

hist, bin_edges = np.histogram(result['MarkovSDVDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['MarkovSDVDelta'] = stats.rv_discrete(name='MarkovSDVDelta', values=(bin_edges[:-1], hist))
result['MarkovSDVDelta'] = cdf['MarkovSDVDelta'].cdf(result['MarkovSDVDelta'].values)

hist, bin_edges = np.histogram(result['ODDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['ODDelta'] = stats.rv_discrete(name='ODDelta', values=(bin_edges[:-1], hist))
result['ODDelta'] = cdf['ODDelta'].cdf(result['ODDelta'].values)

hist, bin_edges = np.histogram(result['KeenerDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerDelta'] = stats.rv_discrete(name='KeenerDelta', values=(bin_edges[:-1], hist))
result['KeenerDelta'] = cdf['KeenerDelta'].cdf(result['KeenerDelta'].values)

hist, bin_edges = np.histogram(result['KeenerORDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerORDelta'] = stats.rv_discrete(name='KeenerORDelta', values=(bin_edges[:-1], hist))
result['KeenerORDelta'] = cdf['KeenerORDelta'].cdf(result['KeenerORDelta'].values)

hist, bin_edges = np.histogram(result['KeenerDRDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerDRDelta'] = stats.rv_discrete(name='KeenerDRDelta', values=(bin_edges[:-1], hist))
result['KeenerDRDelta'] = cdf['KeenerDRDelta'].cdf(result['KeenerDRDelta'].values)

hist, bin_edges = np.histogram(result['KeenerASTDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerASTDelta'] = stats.rv_discrete(name='KeenerASTDelta', values=(bin_edges[:-1], hist))
result['KeenerASTDelta'] = cdf['KeenerASTDelta'].cdf(result['KeenerASTDelta'].values)

hist, bin_edges = np.histogram(result['KeenerTODelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerTODelta'] = stats.rv_discrete(name='KeenerTODelta', values=(bin_edges[:-1], hist))
result['KeenerTODelta'] = cdf['KeenerTODelta'].cdf(result['KeenerTODelta'].values)

hist, bin_edges = np.histogram(result['KeenerSTLDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerSTLDelta'] = stats.rv_discrete(name='KeenerSTLDelta', values=(bin_edges[:-1], hist))
result['KeenerSTLDelta'] = cdf['KeenerSTLDelta'].cdf(result['KeenerSTLDelta'].values)

hist, bin_edges = np.histogram(result['KeenerBLKDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerBLKDelta'] = stats.rv_discrete(name='KeenerBLKDelta', values=(bin_edges[:-1], hist))
result['KeenerBLKDelta'] = cdf['KeenerBLKDelta'].cdf(result['KeenerBLKDelta'].values)

hist, bin_edges = np.histogram(result['KeenerPFDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['KeenerPFDelta'] = stats.rv_discrete(name='KeenerPFDelta', values=(bin_edges[:-1], hist))
result['KeenerPFDelta'] = cdf['KeenerPFDelta'].cdf(result['KeenerPFDelta'].values)

In [37]:
result.head()

,Result,Season,Player,Opponent,PlayerSeed,OpponentSeed,SeedDelta,EfgOPly,EfgDPly,TopOPly,...,MarkovSDVDelta,ODDelta,KeenerDelta,KeenerORDelta,KeenerDRDelta,KeenerASTDelta,KeenerTODelta,KeenerSTLDelta,KeenerBLKDelta,KeenerPFDelta
0,1,2003,1421,1411,16,16,0.516129,0.122845,0.970905,0.945043,...,0.450219,0.397702,0.604486,0.643873,0.678884,0.572757,0.822757,0.579322,0.561816,0.734683
1,0,2003,1421,1400,16,1,0.032258,0.122845,0.970905,0.945043,...,0.077681,0.013676,0.067834,0.111050,0.114880,0.046499,0.371991,0.145514,0.108862,0.235230
2,0,2003,1163,1400,5,1,0.387097,0.581897,0.049569,0.822198,...,0.468271,0.339168,0.480306,0.422867,0.527352,0.462254,0.614333,0.521335,0.664114,0.550875
3,0,2003,1277,1400,7,1,0.322581,0.265086,0.270474,0.918103,...,0.592451,0.419584,0.415208,0.407002,0.451860,0.415755,0.511488,0.486324,0.396608,0.431619
4,0,2003,1345,1400,9,1,0.258065,0.060345,0.437500,0.709052,...,0.334792,0.341904,0.347921,0.279540,0.312910,0.260394,0.323304,0.475930,0.264223,0.298687


### Parameter tunning

| Conf | Result |
|---|---|
| stat+ranking(2x in last 15 days of regular) | 0.5597503677010536 |
|  |  |
|  |  |
|  |  |
|  |  |

In [55]:
(0.56040297536207229+0.56040069470599074+0.56040971897153269+0.56040971897153269+0.5603993445801646)/5

0.5604044905182587

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.regularizers import l2, l1, l1l2
from keras.optimizers import SGD

model = Sequential([
    Dense(43, input_dim=43, W_regularizer=l1l2(0.002, 0.005), init='lecun_uniform'),
    Activation('relu'),
    Dense(43, input_dim=43, W_regularizer=l1l2(0.002, 0.005), init='lecun_uniform'),
    Activation('relu'),
    Dense(43, input_dim=43, W_regularizer=l1l2(0.002, 0.005), init='lecun_uniform'),
    Activation('relu'),
    Dense(1, input_dim=43, W_regularizer=l1l2(0.002, 0.005), init='lecun_uniform'),
    Activation('sigmoid')
])

opt = SGD(lr=0.125, momentum=0.90, decay=4e-5)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_crossentropy'])

Using Theano backend.
WARNING (theano.sandbox.cuda): GPU device gpu0 will be initialized, and used if a GPU is needed. However, no computation, nor shared variables, will be implicitly moved to that device. If you want that behavior, use the 'device' flag instead.
Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN Version is too old. Update to v5, was 3007.)


In [39]:
seg = result[(result['Season'])<2013].copy()
X = seg.ix[:, 6:49].values
Y = seg.ix[:, 0].values
test = result[(result['Season']>=2013)].copy()
Xt = test.ix[:, 6:49].values
Yt = test.ix[:, 0].values

In [40]:
model.fit(X, np.expand_dims(Y, 1), nb_epoch=1000, batch_size=2048, validation_data=(Xt, np.expand_dims(Yt, 1)), verbose=0)

In [42]:
Yp = model.predict_proba(Xt)

 32/536 [>.............................] - ETA: 0s

In [41]:
#keeps the best auc
from sklearn.metrics import log_loss

log_loss(Yt, Yp)

0.55776090372298193

In [43]:
from sklearn.metrics import log_loss
log_loss(Yt, Yp, eps=1e-7)

0.55723111661595859

In [41]:
#keeps the best auc
from sklearn.metrics import roc_auc_score

roc_auc_score(Yt, Yp)

0.78017097349075515

In [44]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Yt, Yp)

0.7856705279572288

using xgboost-lr model

In [107]:
import xgboost as xgb

train_data = xgb.DMatrix(X, label=Y)
valid_data = xgb.DMatrix(Xt, label=Yt)
param = [('lambda', 10), 
         ('alpha', 0),
         ('max_depth', 10),
         ('subsample', 0.8),
         ('eta', 0.08),
         ('objective', 'binary:logistic'), ('eval_metric', 'logloss'), ('eval_metric', 'error'), ('eval_metric', 'auc')]
watchlist = [(valid_data,'eval')]

In [108]:
bst = xgb.train(param, train_data, 100, watchlist, early_stopping_rounds=10)

[0]	eval-logloss:0.679218	eval-error:0.367537	eval-auc:0.71199
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 10 rounds.
[1]	eval-logloss:0.669415	eval-error:0.33209	eval-auc:0.738096
[2]	eval-logloss:0.658011	eval-error:0.322761	eval-auc:0.747953
[3]	eval-logloss:0.647014	eval-error:0.309702	eval-auc:0.754998
[4]	eval-logloss:0.638598	eval-error:0.320896	eval-auc:0.750898
[5]	eval-logloss:0.633694	eval-error:0.328358	eval-auc:0.750842
[6]	eval-logloss:0.628762	eval-error:0.328358	eval-auc:0.753453
[7]	eval-logloss:0.623933	eval-error:0.31903	eval-auc:0.756732
[8]	eval-logloss:0.619459	eval-error:0.309702	eval-auc:0.758952
[9]	eval-logloss:0.613629	eval-error:0.298507	eval-auc:0.764974
[10]	eval-logloss:0.609615	eval-error:0.298507	eval-auc:0.765587
[11]	eval-logloss:0.605318	eval-error:0.304104	eval-auc:0.767675
[12]	eval-logloss:0.602387	eval-error:0.30597	eval-auc:0.766631
[13]	eval-logloss:0.599316	e

# Submit result

In [40]:
submission = pd.read_csv("./Data/sample_submission.csv")
content = []
for row in submission.iterrows():
    content.append(map(int, row[1].ix[0].split('_')))

In [41]:
submission = pd.DataFrame(content, columns=['Season', 'Player', 'Opponent'])

In [42]:
# take features from existing data
t = pd.merge(submission, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t.ix[:, [0,1,2,5]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed']
t = pd.merge(t, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t.ix[:, [0, 1, 2, 3, 6]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
t['SeedDelta'] = (t['OpponentSeed']-t['PlayerSeed']+16.0)/31
t['SeedState'] = (17.0-t['PlayerSeed'].values)/16
t = pd.merge(t, feature, on=['Season', 'Player'])
t = pd.merge(t, feature, left_on=['Season', 'Opponent'], right_on=['Season', 'Player'], suffixes=["Ply", "Opp"])
t.drop('PlayerOpp', 1, inplace=True)
t = t.rename(columns={"PlayerPly": "Player"})

In [44]:
t['MasseyDelta'] = t.MasseyPly-t.MasseyOpp
t.drop(['MasseyPly', 'MasseyOpp'], 1, inplace=True)
t['ColleyDelta'] = t.ColleyPly-t.ColleyOpp
t.drop(['ColleyPly', 'ColleyOpp'], 1, inplace=True)
t['DifferDelta'] = t.DifferPly-t.DifferOpp
t.drop(['DifferPly', 'DifferOpp'], 1, inplace=True)
t['MarkovRDVDelta'] = t.MarkovRDVPly-t.MarkovRDVOpp
t.drop(['MarkovRDVPly', 'MarkovRDVOpp'], 1, inplace=True)
t['MarkovSDVDelta'] = t.MarkovSDVPly-t.MarkovSDVOpp
t.drop(['MarkovSDVPly', 'MarkovSDVOpp'], 1, inplace=True)
t['ODDelta'] = t.ODPly-t.ODOpp
t.drop(['ODPly', 'ODOpp'], 1, inplace=True)
t['KeenerDelta'] = t.KeenerPly-t.KeenerOpp
t.drop(['KeenerPly', 'KeenerOpp'], 1, inplace=True)

In [45]:
t['MasseyDelta'] = cdf['MasseyDelta'].cdf(t['MasseyDelta'].values)
t['ColleyDelta'] = cdf['ColleyDelta'].cdf(t['ColleyDelta'].values)
t['DifferDelta'] = cdf['DifferDelta'].cdf(t['DifferDelta'].values)
t['MarkovRDVDelta'] = cdf['MarkovRDVDelta'].cdf(t['MarkovRDVDelta'].values)
t['MarkovSDVDelta'] = cdf['MarkovSDVDelta'].cdf(t['MarkovSDVDelta'].values)
t['ODDelta'] = cdf['ODDelta'].cdf(t['ODDelta'].values)
t['KeenerDelta'] = cdf['KeenerDelta'].cdf(t['KeenerDelta'].values)

In [47]:
t.drop('SeedState', 1, inplace=True)

In [48]:
X = t.ix[:, 5:41].values
Y = model.predict_proba(X)

8992/9112 [============================>.] - ETA: 0s

In [58]:
Y

array([[ 0.77092475],
       [ 0.23356242],
       [ 0.12581013],
       ..., 
       [ 0.73429012],
       [ 0.67084491],
       [ 0.83557963]], dtype=float32)

In [50]:
t['Prediction']=Y

In [59]:
id = []
pred = []
for row in t.iterrows():
    id.append("{0}_{1}_{2}".format(int(row[1].ix[0]), int(row[1].ix[1]), int(row[1].ix[2])))
    pred.append(row[1].Prediction)

In [60]:
submit = pd.DataFrame({"id":id, "pred":pred}, columns=['id', 'pred'])

In [61]:
submit.head()

,id,pred
0,2013_1103_1107,0.770925
1,2013_1103_1112,0.233562
2,2013_1107_1112,0.125810
3,2013_1103_1125,0.428943
4,2013_1107_1125,0.198940


In [62]:
submit.to_csv("result.csv", index=False)

Try to calculate loss locally

In [55]:
submission = t[['Season', 'Player', 'Opponent', 'Prediction']]

In [54]:
submission.shape

(9112, 4)

In [56]:
t1 = pd.merge(submission, tourney_compact_results, left_on=['Season', 'Player', 'Opponent'], right_on=['Season', 'Wteam', 'Lteam'])
t2 = pd.merge(submission, tourney_compact_results, left_on=['Season', 'Player', 'Opponent'], right_on=['Season', 'Lteam', 'Wteam'])

In [57]:
t1['Label']=1
t2['Label']=0
submission = pd.concat([t1, t2], ignore_index=True)
log_loss(submission['Label'].values, submission['Prediction'].values)

0.55801469215483812